# `TODOIST AUTOMATION`

### `Settings`

#### Libraries

In [ ]:
import requests
import json
import time
import uuid
from datetime import datetime, timedelta, timezone

#### Dependences

In [ ]:
import credentials as cr

###  `Data` 

#### Estraction

In [ ]:
api_token = cr.api_token #SELECT YOUR TODOIST API TOKEN HERE
headers = {"Authorization": f"Bearer {api_token}"}

In [ ]:
headers = {
    "Content-Type": "application/json",
    "X-Request-Id": "un_valor_único_por_cada_solicitud",
    "Authorization": f"Bearer {api_token}"
}

data = {
    "sync_token": "*",
    "resource_types": json.dumps(['items', 'projects'])
}

In [ ]:
response = requests.post("https://api.todoist.com/sync/v9/sync", headers=headers, data=json.dumps(data))
print(response.status_code)

#### Projects

In [ ]:
all_projects = response.json().get('projects', [])
projects_dict={}
for project in all_projects:
    projects_dict[project['id']]=project['name']
projects_dict

#### Tasks

In [ ]:
response.json().keys()

In [ ]:
all_tasks=response.json()["items"]

#### Test

In [ ]:
index=150
print(all_tasks[index]['content'], "-", all_tasks[index]['due']['date'], "-", projects_dict[all_tasks[index]['project_id']])

#### Labels

In [ ]:
label_names=[]
for task in all_tasks:
    gross_labels=task['labels']
    for label in gross_labels:
        if label not in label_names:
            label_names.append(label)
label_names

### `Orders`

#### Creating tasks

In [ ]:
def priority_inversal(n):
    if n==1:
        return 4
    if n==3:
        return 2
    if n==2:
        return 3
    if n==4:
        return 1

In [ ]:
def create_task(name,project_id,date="",priority=4,labels=[]):
    
    priority=priority_inversal(priority)
    
    for label in labels:
        if label not in label_names:
            print("La etiqueta no existe")
            labels=[]

    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_token}',
        'X-Request-Id': str(uuid.uuid4()),
    }

    data = {
        'commands': json.dumps([{
            'type': 'item_add',
            'temp_id': str(uuid.uuid4()),
            'uuid': str(uuid.uuid4()),
            'args': {
                'content': name,
                'project_id': project_id,
                'due': {
                    'date': date
                    },
                'priority': priority,
                'labels': labels
            }
        }])
    }

    response = requests.post('https://api.todoist.com/sync/v9/sync', headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        print("La tarea fue creada exitosamente.")
    else:
        print(f"Error al crear la tarea: {response.status_code}")

In [ ]:
create_task(name = "This is an automated task",
            project_id = 2258455386,
            date = '2024-12-22T09:00:00',
            priority = 4,
            labels = ['Short'])

#### Task tracking (it doesn't work for completed tasks)

In [ ]:
MINUTES=60*60
time_delta = datetime.now(timezone.utc) - timedelta(MINUTES)
timestamp = int(time_delta.timestamp())

data = {
    'sync_token': '*',  # Un asterisco indica una sincronización completa
    'resource_types': json.dumps(['items'])  # Solicita solo las tareas (items)
}

params = {
    'object_type': 'item',
    'event_type': 'completed',
    'since': timestamp,
}

# Realiza la solicitud a la API de Todoist
response = requests.post('https://api.todoist.com/sync/v9/sync', headers=headers, data=json.dumps(data), params = params)

items = response.json().get('items', [])
recently_completed_tasks = [
    item for item in all_tasks if item.get('checked') == 1 and
    datetime.strptime(item['date_completed'], "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc).timestamp() > timestamp
]

# Imprime las tareas completadas recientemente
for task in recently_completed_tasks:
    print(task['content'], task['date_completed'])

In [ ]:
recently_completed_tasks